In [21]:
import time
from huggingface_hub import InferenceClient
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown, clear_output

In [10]:
load_dotenv()
hf_key = os.getenv('HF_KEY')

In [11]:
python_code = """
    def libneiz_formula(iter):
        sum = 0
        for i in range(1,iter+1):
            odd_num = 2*i-1
            if i%2 == 1:
                sum += (1/odd_num)
            else:
                sum -= (1/odd_num)
        return sum*4
"""

In [12]:
# start_time = time.time()
# result = libneiz_formula(100_000_000)
# end_time = time.time()
# execution_time = end_time-start_time

# print(f'The result is {result}')
# print(f'And, the execution time is: {execution_time:.4f} seconds')

In [13]:
# => if you wish to use requests:

# API_URL = "https://router.huggingface.co/v1/chat/completions"
# headers = {
#     "Authorization": f"Bearer {hf_key}",
# }

# messages = [
#     {'role': 'system', 'content': system_prompt},
#     {'role': 'user', 'content': user_prompt}
# ]

# response = requests.post(API_URL, headers=headers, json={
#     "messages": messages,
#     "model": MODEL
# })

system_prompt = """
    Your job is to convert the python code given to you by the user into C++ code. They expect you to return back the C++ code that does exactly the job that their python code does but they expect you C++ code to run faster due to optimized code as well as due to inherent speed of C++ compiler. Just give them the raw code, you do not need to add anything else ok. just give them the functioning code with everthing included for running it.
    Also include the time test to see how long the c++ code will take.
    """
user_prompt = f"""
Can you convert this python code to C++ code:
{python_code}
"""

In [23]:
# Using huggingface_hub's InferenceClient

MODEL = 'deepseek-ai/DeepSeek-V3.1'

client = InferenceClient(
    provider='novita',
    api_key=hf_key
)

# => for non-stream result:

# completion = client.chat.completions.create(
#     model="deepseek-ai/DeepSeek-V3.1",
#     messages=[
#         {
#             "role": "user",
#             "content": "What is the capital of France?"
#         }
#     ],
# )

# print(completion.choices[0].message)

# -> for stream result:

stream = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ],
    stream=True,
)

In [24]:
full_content = ""

for chunk in stream:
    try:
        content = chunk.choices[0].delta.content
        if content:
            full_content += content
            clear_output(wait=True)
            display(Markdown(full_content))
    except (IndexError, AttributeError):
        continue

```cpp
#include <iostream>
#include <chrono>

double libneiz_formula(int iter) {
    double sum = 0.0;
    for (int i = 1; i <= iter; i++) {
        int odd_num = 2 * i - 1;
        if (i % 2 == 1) {
            sum += 1.0 / odd_num;
        } else {
            sum -= 1.0 / odd_num;
        }
    }
    return sum * 4.0;
}

int main() {
    int iterations = 1000000;
    
    auto start = std::chrono::high_resolution_clock::now();
    double result = libneiz_formula(iterations);
    auto end = std::chrono::high_resolution_clock::now();
    
    std::chrono::duration<double> duration = end - start;
    
    std::cout << "Result: " << result << std::endl;
    std::cout << "Time taken: " << duration.count() << " seconds" << std::endl;
    
    return 0;
}
```